In [1]:
# Importació de llibreries per a modelatge, processament de text i validació

import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier  # Classificador Gradient Boosting optimitzat

# Càrrega del conjunt de dades '20 Newsgroups', una col·lecció de textos en anglès classificats per temes

dades = fetch_20newsgroups(subset='all', remove=())  # Es carrega el conjunt complet, sense eliminar cap part

# Es crea un DataFrame amb el text dels documents i la seva etiqueta numèrica
df = pd.DataFrame({
    'text': dades.data,
    'etiqueta_original': dades.target
})

# S’afegeix una columna amb el nom de la categoria textual corresponent a cada etiqueta
noms_categories = dades.target_names
df['tema'] = df['etiqueta_original'].apply(lambda x: noms_categories[x])

# Definició d'una variable binària "esport"
# Classes positives: categories relacionades amb esports i motors

categories_esport = ['rec.sport.baseball', 'rec.sport.hockey', 'rec.autos', 'rec.motorcycles']
df['esport'] = df['tema'].apply(lambda x: 1 if x in categories_esport else 0)

# Separació de les dades (X) i etiquetes binàries (y)

X = df['text'].values
y = df['esport'].values

# Definició d’una llista de paraules buides personalitzada
# Aquestes paraules seran ignorades durant la vectorització

stopwords = [
    'as', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
    'is', 'it', 'for', 'with', 'that', 'this', 'was', 'be',
    'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
    'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not'
]

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Paràmetres a explorar: nombre de temes (k) per a LDA

valors_k = [20]  # En aquest exemple, només es prova amb k = 20

# Inicialització de la llista per emmagatzemar els resultats finals
resultats = []

In [2]:

# Configuració de validació creuada  (5 particions)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Bucle principal: per a cada valor de k (temes), es fa validació creuada
# amb XGBoost entrenat sobre les representacions temàtiques LDA


for k in valors_k:
    puntuacions = []  # Llista per desar els AUCs de cada fold

    for train_idx, test_idx in cv.split(X, y):
        # Separació en conjunt d'entrenament i de prova
        X_train_raw, X_test_raw = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Vectorització dels textos amb CountVectorizer
        # Es limita a les 10.000 paraules més freqüents i es filtren les stopwords

        vectoritzador = CountVectorizer(max_features=10000, stop_words=stopwords)
        X_train_counts = vectoritzador.fit_transform(X_train_raw)
        X_test_counts = vectoritzador.transform(X_test_raw)

        # Reducció de dimensionalitat amb LDA (Latent Dirichlet Allocation)
        # Cada document es representa com una distribució de probabilitats sobre k temes

        lda = LatentDirichletAllocation(n_components=k, random_state=42)
        X_train_topics = lda.fit_transform(X_train_counts)
        X_test_topics = lda.transform(X_test_counts)

        print("iteracio")  # Missatge indicatiu de progrés

        # Entrenament del model XGBoost (Gradient Boosting per classificació binària)
        # Paràmetres comuns: profunditat màxima, nombre d'arbres, mètrica i semilla

        model = XGBClassifier(
            max_depth=6,
            n_estimators=500,
            use_label_encoder=False,  # Es desactiva per evitar warnings antics
            eval_metric='logloss',
            random_state=42
        )
        model.fit(X_train_topics, y_train)
        print("iteracio")
        # Predicció de probabilitats i càlcul de la mètrica AUC-ROC
        # Aquesta mètrica avalua la qualitat de la classificació probabilística

        probabilitats = model.predict_proba(X_test_topics)[:, 1]
        puntuacio = roc_auc_score(y_test, probabilitats)
        puntuacions.append(puntuacio)
        print(puntuacio)  # Impressió de la puntuació AUC del fold actual

    # Emmagatzematge del valor mitjà d'AUC-ROC per a aquest valor de k

    resultats.append({
        'k': k,
        'auc_roc_mitjana': np.mean(puntuacions)
    })
    print(resultats)  # Impressió de resultats acumulats


iteracio


iteracio
0.9640405288040741
iteracio


iteracio
0.9694402047235843
iteracio


iteracio
0.98127959001195
iteracio


iteracio
0.9795027103225513
iteracio


iteracio
0.9772453506913165
[{'k': 20, 'auc_roc_mitjana': np.float64(0.9743016769106954)}]
